<a href="https://colab.research.google.com/github/zahraDehghanian97/stock_market/blob/master/Arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import pytse_client as tse


TRAINING_PERCENTAGE = 0.7
TESTING_PERCENTAGE = 1 - TRAINING_PERCENTAGE
NUMBER_OF_PREVIOUS_DATA_POINTS = 3
LENGTH_DATA_SET = 0
TRAINING_SET_LENGTH = 0
TESTING_SET_LENGTH = 0


def training_testing_buckets(raw_data, training_percentage, testing_percentage):
    global TRAINING_SET_LENGTH, TESTING_SET_LENGTH
    TRAINING_SET_LENGTH = int(LENGTH_DATA_SET * training_percentage)
    TESTING_SET_LENGTH = LENGTH_DATA_SET - TRAINING_SET_LENGTH
    training_set, testing_set = raw_data[0:TRAINING_SET_LENGTH], raw_data[TRAINING_SET_LENGTH:LENGTH_DATA_SET]
    return training_set, testing_set


def evaluate_performance_arima(testing_actual, testing_predict):
    counter = 0
    for i in range(len(testing_actual)-1):
        predict=testing_predict[i+1]- testing_predict[i]
        actual = testing_actual[i+1]-testing_actual[i]
        if (actual > 0 and predict > 0) or (actual < 0 and predict < 0):
            counter+=1
    counter = round((counter / (len(testing_actual)-1))*100,2)
    return counter
    # return mean_squared_error(testing_actual, testing_predict)


def plot_arima(currency, testing_actual, testing_predict, file_name):
    actual = pyplot.plot(testing_actual, label="Actual data points", color="blue")
    testing = pyplot.plot(testing_predict, label="Testing prediction", color="green")
    pyplot.ylabel('currency values for 1 USD')
    pyplot.xlabel('number of days')
    pyplot.title( currency + ' : actual vs predicted ')
    pyplot.legend()
    pyplot.show()
    pyplot.savefig(file_name)
    pyplot.clf()


def load_data_set(currency):
    tse.download(symbols=stock_name)
    ticker = tse.Ticker(stock_name)
    data = ticker.history
    raw_data = data.close.values.tolist()
    global LENGTH_DATA_SET
    LENGTH_DATA_SET = len(raw_data)
    return raw_data


def build_model_predict_arima(training_set, testing_set):
    testing_predict = list()
    training_predict = list(training_set)
    for testing_set_index in range(TESTING_SET_LENGTH):
        arima = ARIMA(training_predict, order=(5, 1, 0))
        arima_model = arima.fit(disp=0)
        forecasting = arima_model.forecast()[0].tolist()[0]
        testing_predict.append(forecasting)
        training_predict.append(testing_set[testing_set_index])
        # print("Predicted = ", testing_predict[-1], "Expected = ", testing_set[testing_set_index])
    print('predicting...')
    print("---------------------------")
    forcast = arima_model.forecast()[0]
    print('The prediction for the next day:', forcast)
    if forcast- testing_set[-1] > 0 : print("last result = +1")
    else :  print("last result = -1")
    print("---------------------------")
    return testing_predict


def arima_model(currency):
    print('\nARIMA Model')

    print('loading the dataset...')
    raw_data = load_data_set(currency)

    print('splitting training and testing set...')
    training_actual_arima, testing_actual_arima = training_testing_buckets(raw_data, TRAINING_PERCENTAGE,
                                                                           TESTING_PERCENTAGE)

    print('building and training model...')
    testing_predict_arima = build_model_predict_arima(training_actual_arima, testing_actual_arima)

    print('evaluating performance...')
    mse_arima = evaluate_performance_arima(testing_actual_arima, testing_predict_arima)
    print('Testing Accuracy: ', mse_arima,"%")

    with open("mse_arima.txt", 'w') as mse_file:
        mse_file.write(str(mse_arima) + '\n')
    print("------------------------")
    print('plotting the graph...')
    plot_arima(currency, testing_actual_arima, testing_predict_arima, "testing_prediction_arima.pdf")

    print('done...')
    return raw_data, testing_predict_arima


if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    stock_name = input('Enter stock name:')
    # stock_name = 'Ø®Ú†Ø±Ø®Ø´'
    arima_model(stock_name)  # setting the entry point


SystemError: ignored

In [ ]:
pip install pytse_client

     |████████████████████████████████| 122kB 10.1MB/s 
  Created wheel for jdatetime: filename=jdatetime-3.6.2-cp36-none-any.whl size=11783 sha256=b8524c351cac12afeb10ead4e877c526446b081652cb7da984653b51c63432bd
  Stored in directory: /root/.cache/pip/wheels/ea/7d/d2/92961c39b79a0556bc4ce7c20185fdab84969ecb8fe2e5e8cc
Successfully built jdatetime
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
